In [131]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [132]:
url = 'https://fightingillini.com/sports/2021/4/30/mens-basketball-history'


def set_first_row_as_header(df):
    df = df.copy()  # avoid modifying original
    df.columns = df.iloc[0]
    return df[1:].reset_index(drop=True)

def try_fix_header(df):
    if df.shape[0] > 1:
        return set_first_row_as_header(df)
    return df


# Scrape the webpage
res = requests.get(url)
soup = BeautifulSoup(res.text, 'html.parser')



all_tables = soup.find_all("table", class_="release")
titles_dict = {}

for table in all_tables:
    rows = table.find_all("tr")
    if not rows:
        continue

    # Try to extract a title from the first <th>
    header_cells = rows[0].find_all("th")
    if not header_cells:
        continue

    title = header_cells[0].get_text(strip=True).replace('\xa0', ' ')

    # Grab all rows
    raw_rows = []
    for tr in table.find_all("tr"):
        cols = [cell.get_text(strip=True) for cell in tr.find_all(["td", "th"])]
        if cols:
            raw_rows.append(cols)

    # Skip section title row (first) if it has only one column
    if len(raw_rows) > 1 and len(raw_rows[0]) == 1:
        raw_rows = raw_rows[1:]

    # Create DataFrame
    df = pd.DataFrame(raw_rows)

 

    # Save to dictionary
    titles_dict[title] = df


In [133]:
titles_dict

{'1* National Title':       0
 0  1915,
 '5 NCAA Final Fours':                               0
 0  1949, 1951, 1952, 1989, 2005,
 '35* NCAA Tournament Appearances':                                                    0
 0  1942, 1949, 1951, 1952, 1963, 1981, 1983, 1984...,
 '18* Big Ten Regular Season Championships':                                                    0
 0  1915, 1917, 1924, 1935, 1937, 1942, 1943, 1949...,
 '4 Big Ten Tournament Championships':                         0
 0  2003, 2005, 2021, 2024,
 'Winningest Seasons – By Total Wins':      0   1     2
 0   1.  37  2005
 1   2.  31  1989
 2   3.  29  2024
 3   4.  27  2001
 4   5.  26  2006
 5       26  2004
 6       26  2002
 7       26  1985
 8       26  1984
 9  10.  25  2003,
 'Winningest Seasons – By Win Percentage':      0      1     2     3
 0   1.  1.000  16-0  1915
 1   2.   .949  37-2  2005
 2   3.   .944  17-1  1943
 3   4.   .861  31-5  1989
 4   5.   .846  22-4  1952
 5   6.   .840  21-4  1949
 6   7.   .83

In [134]:
titles_dict['1* National Title']

,0
0,1915


In [135]:
titles_dict['Winningest Seasons – By Total Wins']

,0,1,2
0,1.,37,2005
1,2.,31,1989
2,3.,29,2024
3,4.,27,2001
4,5.,26,2006
5,,26,2004
6,,26,2002
7,,26,1985
8,,26,1984
9,10.,25,2003
